battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price \* (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [ ]:
import pyomo.environ as pyo
from pyomo.util.check_units import assert_units_consistent
from pyomo.environ import units

units.load_definitions_from_strings(["USD = [currency]"])

In [ ]:
PEAK_PRICE = 0.5
""" https://www.pge.com/assets/pge/docs/account/rate-plans/residential-electric-rate-plan-pricing.pdf """

OFF_PEAK_PRICE = 0.4
""" https://www.pge.com/assets/pge/docs/account/rate-plans/residential-electric-rate-plan-pricing.pdf """

BATTERY_COST_PER_KW = 0.15
"""
battery levelized cost per kW-hour
https://www.lazard.com/research-insights/levelized-cost-of-energyplus-lcoeplus/
"""

PEAK_CONSUMPTION = 10
"""
https://www.constellation.com/energy-101/energy-education/average-home-power-usage.html#how-many-kwh-house-use
"""

OFF_PEAK_CONSUMPTION = 20
"""
https://www.constellation.com/energy-101/energy-education/average-home-power-usage.html#how-many-kwh-house-use

"""

In [ ]:
SOLAR_INSTALLATION_SIZES = {
    3: 0.282,
    5: 0.250,
    6: 0.230,
    8: 0.210,
    10: 0.190,
    12: 0.117,
}
"""
Maps solar installation sizes (in kW) to their corresponding costs levelized cost per kWh.

https://www.integratesun.com/post/how-much-does-it-cost-to-install-solar-panels-in-2025-1
"""

In [ ]:
model = pyo.ConcreteModel()


model.peak_load = pyo.Param(initialize=PEAK_CONSUMPTION, units=units.kWh)
model.off_peak_load = pyo.Param(initialize=OFF_PEAK_CONSUMPTION, units=units.kWh)
model.peak_grid_price = pyo.Param(initialize=PEAK_PRICE, units=units.USD / units.kWh)
model.off_peak_grid_price = pyo.Param(
    initialize=OFF_PEAK_PRICE, units=units.USD / units.kWh
)
model.battery_cost_per_kw = pyo.Param(
    initialize=BATTERY_COST_PER_KW, units=units.USD / units.kWh
)


model.SOLAR_SIZES = pyo.Set(initialize=SOLAR_INSTALLATION_SIZES.keys())
model.solar_size_flags = pyo.Var(
    model.SOLAR_SIZES, within=pyo.Binary, units=units.USD / units.kWh
)
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals, units=units.kWh)
model.solar_cost = pyo.Var(within=pyo.NonNegativeReals, units=units.USD / units.kWh)

model.peak_grid_consumption = pyo.Var(within=pyo.NonNegativeReals, units=units.kWh)
model.off_peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals, units=units.kWh)
model.battery_capacity = pyo.Var(
    within=pyo.NonNegativeReals, bounds=(0, 4), units=units.kWh
)

model.minimize_cost = pyo.Objective(
    expr=(model.peak_grid_price * model.peak_grid_consumption)
    + (model.off_peak_grid_price * model.off_peak_grid_usage)
    + (model.solar_capacity * model.solar_cost)
    + (model.battery_cost_per_kw * model.battery_capacity),
    sense=pyo.minimize,
)


model.off_peak_constraint = pyo.Constraint(
    expr=model.off_peak_load <= model.off_peak_grid_usage + model.battery_capacity
)

model.peak_constraint = pyo.Constraint(
    expr=model.peak_load <= model.peak_grid_consumption + model.solar_capacity
)

model.battery_charging_constraint = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)


def solar_cost_constraint(model):
    """set the solar cost based on the selected installation size"""
    sum_term = sum(
        SOLAR_INSTALLATION_SIZES[i] * model.solar_size_flags[i]
        for i in model.SOLAR_SIZES
    )
    return model.solar_cost == sum_term


model.solar_cost_constraint = pyo.Constraint(rule=solar_cost_constraint)


def solar_capacity_constraint(model):
    return model.solar_capacity == sum(
        size * model.solar_size_flags[size] for size in model.SOLAR_SIZES
    )


model.solar_capacity_constraint = pyo.Constraint(rule=solar_capacity_constraint)


model.sos1_constraint = pyo.SOSConstraint(var=model.solar_size_flags, sos=1)


assert_units_consistent(model.off_peak_constraint)
assert_units_consistent(model.solar_cost_constraint)
assert_units_consistent(model.peak_constraint)
assert_units_consistent(model.battery_charging_constraint)
# assert_units_consistent(model.solar_capacity_constraint)
assert_units_consistent(model.minimize_cost)
optimizer = pyo.SolverFactory("gurobi")
optimizer.solve(model, tee=True)
print(
    f"solar capacity: {model.solar_capacity.value} kW, battery capacity: {model.battery_capacity.value} kWh, off peak grid usage: {model.off_peak_grid_usage.value} kW, peak grid consumption: {model.peak_grid_consumption.value} kW"
)
print(model.display())